In [1]:
import os 
import regex
from tqdm import tqdm 
from datasets import load_dataset

# Clean and preprocess

In [27]:
base_folder = "./data/archive/output"
output_folder = "./data_cleaned"

# 6 last sentences 

In [42]:
def split_sentence(comment):
    expression = r'(?<=[.!?])\s+'
    splitted = regex.split(expression, comment)

    res = []

    for sentence in splitted[0:-8:1]:
        if sentence != "":
            sentence = sentence.strip()
            sentence = clean_url(sentence)
            sentence = clean_general(sentence)

            if sentence == "<illegal>" or sentence == "<empty>": 
                continue

            res.append(sentence)

    return res

def clean_general(comment):
    illegal_content = regex.compile(r"^[\?\.\#\@\-\+\=\$\^\&,]+$")
    if comment == "" or comment == "\n":
        return "<empty>"
    elif len(regex.findall(illegal_content, comment)) != 0:
        return "<illegal>"
    else:
        return comment

def clean_url(comment):
    expression = regex.compile(
        r"(?:link.*?)?https?:\/\/[\w\-\.\/\@\?\#\:\=\&\%]+")
    matches = regex.findall(expression, comment)

    for word in matches:
        comment = comment.replace(word, "<link> ")

    return comment

def save_files(text_list, output_path): 
    with open(output_path, "w", encoding="utf-8") as f: 
        for i in range(len(text_list)): 
            f.write(f"{text_list[i]}\n")

In [55]:
output_file_counter = 0
for input_file in tqdm(os.listdir(base_folder)): 
    with open(os.path.join(base_folder, input_file), "r", encoding="utf-8") as fi: 
        line_list = fi.readlines()

        content_counter = None
        max_length = 0

        for i, text in enumerate(line_list): 
            if len(text) >= max_length: 
                max_length = max(max_length, len(text))
                content_counter = i 
            
        content = line_list[content_counter]
        text_list = split_sentence(content) 

        output_path = f"./data_cleaned/cleaned_{str(output_file_counter).zfill(5)}.txt"

        with open(output_path, "w", encoding="utf-8") as f: 
            for i in range(len(text_list)): 
                f.write(f"{text_list[i]}\n")
        f.close()
            
        output_file_counter += 1

100%|██████████| 10415/10415 [00:45<00:00, 230.98it/s]


# Convert to arrow

In [3]:
dataset = load_dataset("./data_cleaned", split="train")

Resolving data files:   0%|          | 0/10415 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
dataset.save_to_disk("./data_final")

Saving the dataset (0/1 shards):   0%|          | 0/2193889 [00:00<?, ? examples/s]